In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64
import urllib.parse

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from AnimalShelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = urllib.parse.quote_plus('aacuser')
password = urllib.parse.quote_plus('!P@ssw0rd!')

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

# define global variables for button usage
global b1
b1 = int(0)
global b2
b2 = int(0)
global b3
b3 = int(0)
global b4
b4 = int(0)


## Debug
print(len(df.to_dict(orient='records')))
print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))),
    html.Center("004a27d2-7a89-4ab3-afc7-de56cb281e35"),
    html.Center(html.B(html.H1('Palmer CS-340 Dashboard'))),
    html.Hr(),
    html.Div(className='buttonRow',
        style={'display' : 'flex'},
            children=[
                html.Button(id='submit-button-one', n_clicks=0, children='Water Rescue'),
                html.Button(id='submit-button-two', n_clicks=0, children='Mountain or Wilderness'),
                html.Button(id='submit-button-three', n_clicks=0, children='Disaster Rescue or Individual Tracking'),
                html.Button(id='submit-button-four', n_clicks=0, children='Reset')
            ]),
    html.Hr(),
    dash_table.DataTable(id='datatable-id',
                        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                        data=df.to_dict('records'),
                        editable=False,
                        filter_action="native",
                        sort_action="native",
                        sort_mode="multi",
                        column_selectable=False,
                        row_selectable="single",
                        row_deletable=False,
                        selected_columns=[],
                        selected_rows=[0],
                        page_action="native",
                        page_current=0,
                        page_size=10
                        ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            ),
    #html.Br(),
    #html.Hr(),
    # This sets up the output element for the dashboard. The
    # purpose of the stlye option is to make sure that the 
    # output will function like a regular text area and accept
    # newline ('\n') characters as line-breaks.
    html.Div(id="query-out", style={'whiteSpace': 'pre-line'}),
    #TODO: insert unique identifier code here. Please Note: 
    # when you insert another HTML element here, you will need to 
    # add a comma to the previous line.  
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################
 

@app.callback(Output('datatable-id','data'),
            [Input('submit-button-one', 'n_clicks'),
            Input('submit-button-two', 'n_clicks'),
            Input('submit-button-three', 'n_clicks'),
            Input('submit-button-four', 'n_clicks')
            ])

def update_dashboard(button1, button2, button3, button4):
    # start case
    df = pd.DataFrame.from_records(db.read({}))
    # define b1,b2,b3,b4 as global before reference
    global b1
    global b2
    global b3
    global b4

    # use higher number of button clicks to determine filter type, can you think of a better way? ...
    if (button1 != b1):
        df = pd.DataFrame.from_records(db.read({"breed":{"$in":["Labrador Retriever Mix","Chesapeake Bay Retriever","Newfoundland"]},"sex_upon_outcome":"Intact Female","age_upon_outcome_in_weeks":{"$gte":26},"age_upon_outcome_in_weeks":{"$lte":156}}))
        # increment b1
        b1 = button1
    if (button2 != b2):
        df = pd.DataFrame.from_records(db.read({"breed":{"$in":["German Shepherd","Alaskan Malamute","Old English Sheepdog","Siberian Husky","Rottweiler"]},"sex_upon_outcome":"Intact Male","age_upon_outcome_in_weeks":{"$gte":26},"age_upon_outcome_in_weeks":{"$lte":156}}))
        # increment b2
        b2 = button2
    if (button3 != b3):
        df = pd.DataFrame.from_records(db.read({"breed":{"$in":["Doberman Pinscher","German Shepherd","Golden Retriever","Bloodhound","Rottweiler"]},"sex_upon_outcome":"Intact Male","age_upon_outcome_in_weeks":{"$gte":20},"age_upon_outcome_in_weeks":{"$lte":300}}))
        # increment b3
        b3 = button3
    if (button4 != b4):
        df = pd.DataFrame.from_records(db.read({}))
        # increment b4
        b4 = button4
            
    # Cleanup Mongo _id field
    df.drop(columns=['_id'],inplace=True)
    update_graphs(df)
    return df.to_dict('records')


# Display the breeds of animal based on quantity represented in
# the data table

@app.callback(Output('graph-id',"children"),
            [Input('datatable-id',"derived_virtual_data")
            ])

def update_graphs(viewData):
    return [dcc.Graph(figure = px.pie(viewData, names='breed', title='Preferred Animals'))]

#This callback will highlight a cell on the data table when the user selects it
@app.callback(Output('datatable-id', 'style_data_conditional'),
            [Input('datatable-id', 'selected_columns')
            ])
    
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if viewData is None:
        return
    elif index is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[dff.iloc[row,12],dff.iloc[row,13]],zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,12],dff.iloc[row,13]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]



app.run_server(debug=True)


10002
Index(['rec_num', 'age_upon_outcome', 'animal_id', 'animal_type', 'breed',
       'color', 'date_of_birth', 'datetime', 'monthyear', 'name',
       'outcome_type', 'sex_upon_outcome', 'location_lat', 'location_long',
       'age_upon_outcome_in_weeks', 'outcome_subtype'],
      dtype='object')
Dash app running on http://127.0.0.1:15625/
